In [27]:
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.converters import QuadraticProgramToQubo

from qiskit import IBMQ, Aer
from qiskit.algorithms import QAOA, VQE, NumPyMinimumEigensolver
from qiskit.utils import QuantumInstance, algorithm_globals
from qiskit.providers.aer.noise.noise_model import NoiseModel

from qiskit_optimization.algorithms import MinimumEigenOptimizer

In [28]:
def energy_cost_quadratic_program():
    ### Define crop-yield quadratic coefficients

    max_source_cost = 5 #Pmax
    max_state = 1
    energy_cost_estimation = 10

    # parameters
    cost = ["source", "state"]

    # Coefficients: Ai = 2000, Bi=5, Ci=10
    # linear dependencies
    linear_cost = {
        "source": 5,
        "state": 2,
    }

    # quadratic dependencies
    quadratic_cost = {("source", "source"): 1}
    
    # Create a QuadraticProgram
    energy_cost = QuadraticProgram(
        name="Energy Cost",
    )

    # Add crop-yield variables
    energy_cost.integer_var(name="source", lowerbound=1, upperbound=max_source_cost)
    energy_cost.integer_var(name="state", lowerbound=1, upperbound=max_state)
    
    # Add crop-yield quadratic using monoculture and intercrop yield variables
    energy_cost.minimize(
        linear=linear_cost,
        quadratic=quadratic_cost
    )

    # Add constraint for the total farm area
    energy_cost.linear_constraint(
        sense="<=",
        rhs=energy_cost_estimation,
        name="Energy cost",
    )
    return energy_cost

In [29]:
energy_cost = energy_cost_quadratic_program()

In [30]:
convert_to_qubo = QuadraticProgramToQubo()
qubo_energy_cost = convert_to_qubo.convert(energy_cost)

In [31]:
op, offset = qubo_energy_cost.to_ising()
print('offset: {}'.format(offset))
print('operator:')
print(op)

offset: 1490.0
operator:
675.0 * ZIIIIIII
+ 900.0 * IZIIIIII
+ 270.0 * ZZIIIIII
+ 450.0 * IIZIIIII
+ 135.0 * ZIZIIIII
+ 180.0 * IZZIIIII
+ 225.0 * IIIZIIII
+ 67.5 * ZIIZIIII
+ 90.0 * IZIZIIII
+ 45.0 * IIZZIIII
- 5.5 * IIIIIZII
- 11.0 * IIIIIIZI
+ 1.0 * IIIIIZZI
- 5.5 * IIIIIIIZ
+ 0.5 * IIIIIZIZ
+ 1.0 * IIIIIIZZ


In [32]:
# Estimate the number of qubits required
print(f"Number of qubits required is {op.num_qubits}")

Number of qubits required is 8


<h1> QAOA solution </h1>

In [16]:
backend = Aer.get_backend("qasm_simulator")
algorithm_globals.random_seed = 271828

In [17]:
def get_QAOA_solution_for(
    quadprog: QuadraticProgram,
    quantumInstance: QuantumInstance,
):
    _eval_count = 0

    def callback(eval_count, parameters, mean, std):
        nonlocal _eval_count
        _eval_count = eval_count

    # Create solver
    solver = QAOA(
        quantum_instance=quantumInstance,
        callback=callback,
    )
 
    # Create optimizer for solver
    optimizer = MinimumEigenOptimizer(solver)

    # Get result from optimizer
    result = optimizer.solve(quadprog)
    return result, _eval_count

In [33]:
# Create a QuantumInstance
simulator_instance = QuantumInstance(
    backend=backend,
    seed_simulator=algorithm_globals.random_seed,
    seed_transpiler=algorithm_globals.random_seed,
)

# Get QAOA result
qaoa_result, qaoa_eval_count = get_QAOA_solution_for(energy_cost, simulator_instance)

# Format and print result
print("Solution found using the QAOA method:\n")
print(f"Maximum energy generated is {qaoa_result.fval} Xhour")
print(f"Sources used are: ")
for source_contribution, name_contribution in zip(qaoa_result.x, qaoa_result.variable_names):
    print(f"\t{source_contribution} (coeff) of {name_contribution}")

print(f"\nThe solution was found within {qaoa_eval_count} evaluations of QAOA.")

Solution found using the QAOA method:

Maximum energy generated is 8.0 Xhour
Sources used are: 
	1.0 (coeff) of source
	1.0 (coeff) of state

The solution was found within 16 evaluations of QAOA.


<h1> Classical solution </h1>

In [19]:
def get_classical_solution_for(quadprog: QuadraticProgram):
    # Create solver
    solver = NumPyMinimumEigensolver()

    # Create optimizer for solver
    optimizer = MinimumEigenOptimizer(solver)

    # Return result from optimizer
    return optimizer.solve(quadprog)

In [34]:
# Get classical result
classical_result = get_classical_solution_for(energy_cost)

# Format and print result
print("Solution found using the classical method:\n")
print(f"Maximum energy generated is {classical_result.fval} Xhour")
print(f"Sources used are: ")

_sources = [v.name for v in energy_cost.variables]
for source_index, source_contribution in enumerate(classical_result.x):
    print(f"\t{source_contribution} (coeff) of {_sources[source_index]}")

Solution found using the classical method:

Maximum energy generated is 8.0 Xhour
Sources used are: 
	1.0 (coeff) of source
	1.0 (coeff) of state


<h1> VQE solution </h1>

In [35]:
def get_VQE_solution_for(
    quadprog: QuadraticProgram,
    quantumInstance: QuantumInstance,
):
    _eval_count = 0

    def callback(eval_count, parameters, mean, std):
        nonlocal _eval_count
        _eval_count = eval_count

    # Create solver and optimizer
    solver = VQE(quantum_instance=quantumInstance, callback=callback)

    # Create optimizer for solver
    optimizer = MinimumEigenOptimizer(solver)

    # Get result from optimizer
    result = optimizer.solve(quadprog)
    return result, _eval_count

In [36]:
# Create a QuantumInstance
simulator_instance = QuantumInstance(
    backend=backend,
    seed_simulator=algorithm_globals.random_seed,
    seed_transpiler=algorithm_globals.random_seed,
)

# Get VQE result
vqe_result, vqe_eval_count = get_VQE_solution_for(energy_cost, simulator_instance)

# Format and print result
print("Solution found using the VQE method:\n")
print(f"Maximum energy generated is {vqe_result.fval} Xhour")
print(f"Sources used are: ")
for source_contribution, source_name in zip(vqe_result.x, vqe_result.variable_names):
    print(f"\t{source_contribution} (coeff) of {source_name}")

print(f"\nThe solution was found within {vqe_eval_count} evaluations of VQE")

Solution found using the VQE method:

Maximum energy generated is 8.0 Xhour
Sources used are: 
	1.0 (coeff) of source
	1.0 (coeff) of state

The solution was found within 33 evaluations of VQE


<h1> Run on a fake backend </h1>

In [23]:
import qiskit.test.mock as Fake
fake_device = Fake.FakeJohannesburg()

In [24]:
# Create the noise model, which contains the basis gate set
noise_model = NoiseModel.from_backend(fake_device)

# Get the coupling map
coupling_map = fake_device.configuration().coupling_map

In [25]:
fake_instance = QuantumInstance(
    backend=backend,
    basis_gates=noise_model.basis_gates,
    coupling_map=coupling_map,
    noise_model=noise_model,
    seed_simulator=algorithm_globals.random_seed,
    seed_transpiler=algorithm_globals.random_seed,
)

In [37]:
# Get QAOA result
qaoa_result, qaoa_eval_count = get_QAOA_solution_for(energy_cost, fake_instance)

# Format and print result
print("Solution found using the QAOA method:\n")
print(f"Maximum energy generated is {qaoa_result.fval} Xhour")
print(f"Sources used are: ")
for source_contribution, source_name in zip(qaoa_result.x, qaoa_result.variable_names):
    print(f"\t{source_contribution} ha of {source_name}")

print(f"\nThe solution was found within {qaoa_eval_count} evaluations of QAOA.")

Solution found using the QAOA method:

Maximum energy generated is 8.0 Xhour
Sources used are: 
	1.0 ha of source
	1.0 ha of state

The solution was found within 3 evaluations of QAOA.


<h1> Running on a real quantum computer </h1>

In [ ]:
IBMQ.load_account()
provider = IBMQ.get_provider(hub="ibm-q")
backend = provider.get_backend("ibmq_guadalupe")

In [ ]:
quantum_instance = QuantumInstance(backend)

In [ ]:
result = get_VQE_solution_for(qubo_energy_cost,quantum_instance)